In [1]:
import numpy as np
import pandas as pd
import os 
from PIL import Image
from glob import glob   # takes care of various extension of image files

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset

from sklearn.utils import shuffle

In [3]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
    print('gpu')
else:
    device=torch.device('cpu')
    print('cpu')

gpu


### Generating Datasets

In [35]:
# Creating child class 'BrainMRIDataset' from parent class 'Dataset'
class BrainMRIDataset(Dataset):

    def __init__(self, data_dir, reshape=True, height=128, width=128, autoencoder=False):
        self.data_dir = data_dir
        # No Brain Tumor MRIDataset
        self.no_class = glob(data_dir+'/no/*')
        # Yes Brain MRIDataset
        self.yes_class = glob(data_dir+'/yes/*')

        self.height = height
        self.width = width
        self.reshape = reshape
        self.autoencoder = autoencoder

        # return 0 if no tumor
        labels = [0 for i in range(len(self.no_class))]
        # return 1 if tumor exists
        labels += [1 for i in range(len(self.yes_class))]

        image_links = self.no_class + self.yes_class

        self.df = pd.DataFrame({'images': image_links, 'labels': labels})
        self.df.reset_index(inplace=True, drop=True)

    def __len__(self):
        return len(self.no_class)+len(self.yes_class)

    def __getitem(self, idx):
        image_list = self.df['image'][idx]
        label_list = self.df['labels'][idx]

        if type(image_list) == str:
            image_list = [image_list]

        if not isinstance(label_list.values,np.int64):
            label_list = label_list.values

        image_array = []
        
        for image in image_list:
            image = Image.open(image).convert("L")

            if self.reshape:
                image = image.resize((self.height,self.width))

        array = np.asarray(image)
        array = array.reshape(1,self.height,self.width)
        image_array.append(array)

        return [torch.tensor(image_array,device=device),
                torch.tensor(label_list,device=device)]

    def __repr__(self):
        return str(self.df.sample(10))

In [39]:

dataset = BrainMRIDataset("brain_tumor_dataset")

In [40]:
print(dataset)

                               images  labels
183  brain_tumor_dataset/yes\Y257.jpg       1
130  brain_tumor_dataset/yes\Y157.JPG       1
172  brain_tumor_dataset/yes\Y247.JPG       1
179  brain_tumor_dataset/yes\Y253.JPG       1
234   brain_tumor_dataset/yes\Y77.jpg       1
252   brain_tumor_dataset/yes\Y99.JPG       1
134  brain_tumor_dataset/yes\Y160.JPG       1
6    brain_tumor_dataset/no\15 no.jpg       0
214   brain_tumor_dataset/yes\Y54.jpg       1
54     brain_tumor_dataset/no\N19.JPG       0
